# NSS Tutorial

Field	| Type
--------|-------
ukprn	| varchar(8)
institution	| varchar(100)
subject	| varchar(60)
level	| varchar(50)
question	| varchar(10)
A_STRONGLY_DISAGREE	| int(11)
A_DISAGREE	| int(11)
A_NEUTRAL	| int(11)
A_AGREE	| int(11)
A_STRONGLY_AGREE	| int(11)
A_NA	| int(11)
CI_MIN	| int(11)
score	| int(11)
CI_MAX	| int(11)
response	| int(11)
sample	| int(11)
aggregate	| char(1)

National Student Survey 2012

The National Student Survey <http://www.thestudentsurvey.com/> is presented to thousands of graduating students in UK Higher Education. The survey asks 22 questions, students can respond with STRONGLY DISAGREE, DISAGREE, NEUTRAL, AGREE or STRONGLY AGREE. The values in these columns represent PERCENTAGES of the total students who responded with that answer.

The table `nss` has one row per institution, subject and question.

In [1]:
import findspark
import pandas as pd
findspark.init()

SVR = '192.168.31.31'
from pyspark.sql import SparkSession

sc = (SparkSession.builder.appName('app08+') 
      .master(f'spark://{SVR}:7077') 
      .config('spark.sql.warehouse.dir', f'hdfs://{SVR}:9000/user/hive/warehouse') 
      .config('spark.cores.max', '4') 
      .config('spark.executor.instances', '1') 
      .config('spark.executor.cores', '2') 
      .config('spark.executor.memory', '10g') 
      .enableHiveSupport().getOrCreate())

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


## 1. Check out one row

The example shows the number who responded for:

- question 1
- at 'Edinburgh Napier University'
- studying '(8) Computer Science'

**Show the the percentage who STRONGLY AGREE**

In [2]:
nss = sc.read.table('sqlzoo.nss')

In [3]:
(nss.filter((nss['question']=='Q01') & 
            (nss['institution']=='Edinburgh Napier University') & 
            (nss['subject']=='(8) Computer Science'))
 .select('A_STRONGLY_AGREE')
 .toPandas())

,A_STRONGLY_AGREE
0,23


## 2. Calculate how many agree or strongly agree

**Show the institution and subject where the score is at least 100 for question 15.**

In [4]:
(nss.filter((nss['question']=='Q15') & (nss['score']>=100))
 .select('institution', 'subject')
 .toPandas())

,institution,subject
0,Kingston College,(I) Education
1,"Royal Holloway, University of London",(L) Geographical Studies
2,Solihull College,(I) Education
3,Stafford College,(D) Business and Administrative studies
4,University of Southampton,(E) Mass Communications and Documentation
5,University of Wolverhampton,(7) Mathematical Sciences
6,University of Leicester,(2) Subjects allied to Medicine
7,University of Newcastle upon Tyne,(E) Mass Communications and Documentation
8,"Bishop Grosseteste University College, Lincoln",(F) Languages
9,Universities of East Anglia and Essex; Joint P...,(G) Historical and Philosophical studies


## 3. Unhappy Computer Students

**Show the institution and score where the score for '(8) Computer Science' is less than 50 for question 'Q15'**

In [5]:
(nss.filter((nss['question']=='Q15') & 
           (nss['subject']=='(8) Computer Science') & 
           (nss['score']<50))
 .select('institution', 'score')
 .toPandas())

,institution,score
0,Blackburn College,45
1,North Lindsey College,30
2,Plymouth College of Art,47
3,Somerset College of Arts and Technology,48
4,"University of Wales, Newport",30
5,Universities of East Anglia and Essex; Joint P...,45


## 4. More Computing or Creative Students?

**Show the subject and total number of students who responded to question 22 for each of the subjects '(8) Computer Science' and '(H) Creative Arts and Design'.**

> _HINT_    
> You will need to use SUM over the response column and GROUP BY subject

In [6]:
(nss.filter((nss['question']=='Q22') & 
            (nss['subject'].isin(
                ['(8) Computer Science', '(H) Creative Arts and Design'])))
 .groupBy('subject')
 .agg({'response': 'sum'})
 .toPandas())

,subject,sum(response)
0,(8) Computer Science,10252
1,(H) Creative Arts and Design,33336


## 5. Strongly Agree Numbers

**Show the subject and total number of students who A_STRONGLY_AGREE to question 22 for each of the subjects '(8) Computer Science' and '(H) Creative Arts and Design'.**

> _HINT_    
> The A_STRONGLY_AGREE column is a percentage. To work out the total number of students who strongly agree you must multiply this percentage by the number who responded (response) and divide by 100 - take the SUM of that.

In [7]:
from pyspark.sql.functions import *
(nss.withColumn('n_strongly_agree', nss['response']*nss['A_STRONGLY_AGREE']/100)
     .filter((nss['question']=='Q22') &
             (nss['subject'].isin(
                 ['(8) Computer Science', '(H) Creative Arts and Design'])))
    .select('subject', col('n_strongly_agree').cast('int'))
    .groupBy('subject')
    .sum()
    .toPandas())

,subject,sum(n_strongly_agree)
0,(8) Computer Science,3355
1,(H) Creative Arts and Design,11992


## 6. Strongly Agree, Percentage

**Show the percentage of students who A_STRONGLY_AGREE to question 22 for the subject '(8) Computer Science' show the same figure for the subject '(H) Creative Arts and Design'.**

Use the **ROUND** function to show the percentage without decimal places.

In [8]:
(nss.withColumn('n_sa', nss['A_STRONGLY_AGREE']*nss['response'])
    .filter((nss['question']=='Q22') &
            (nss['subject'].isin(
                ['(8) Computer Science', '(H) Creative Arts and Design'])))
    .select('subject', 'n_sa', 'response')
    .groupBy('subject')
    .sum()
    .withColumn('pct', round(col('sum(n_sa)')/col('sum(response)'), 0))
    .select('subject', 'pct')
    .toPandas())

,subject,pct
0,(8) Computer Science,33.0
1,(H) Creative Arts and Design,36.0


## 7. Scores for Institutions in Manchester

**Show the average scores for question 'Q22' for each institution that include 'Manchester' in the name.**

The column **score** is a percentage - you must use the method outlined above to multiply the percentage by the **response** and divide by the total response. Give your answer rounded to the nearest whole number.

In [9]:
(nss.withColumn('score', nss['response']*nss['score'])
    .filter((nss['question']=='Q22') & (nss['institution'].contains('Manchester')))
    .select('institution', 'score', 'response')
    .groupBy('institution')
    .sum()
    .withColumn('score', round(col('sum(score)')/col('sum(response)'), 0))
    .select('institution', 'score')
    .toPandas())

,institution,score
0,Manchester Metropolitan University,81.0
1,University of Manchester,83.0
2,The Manchester College,72.0


## 8.Number of Computing Students in Manchester

**Show the institution, the total sample size and the number of computing students for institutions in Manchester for 'Q01'.**

In [10]:
(nss.filter((nss['question']=='Q01') & 
            (nss['institution'].contains('Manchester')))
 .select('institution', 'sample', 'subject',
        when(nss['subject']=='(8) Computer Science', nss['sample'])
        .otherwise(0).alias('comp'))
 .groupBy('institution')
 .sum()
 .toPandas())

,institution,sum(sample),sum(comp)
0,Manchester Metropolitan University,6994,310
1,University of Manchester,8065,180
2,The Manchester College,537,46


In [11]:
sc.stop()